In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [10]:
STORM_NUMBER = 7
TITLE = "1-2 February 2022"
SUBTITLE = "8 AM 3 February 2022"
sts = utc(2022, 2, 2, 6)
ets = utc(2022, 2, 4, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [5]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [16]:
cull = ['3OI']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['IA-AP-8', 0.2],
    ['MTAI4', 0],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=6)
res = mp.postprocess(filename='220203.png')
mp.close()

     state  wfo   val       lon      lat                              geo  \
0       CO  PUB  22.0 -104.8400  38.7400  POINT (-420114.074 -683522.480)   
1       CO  PUB  22.0 -104.8800  38.7400  POINT (-423581.194 -683319.830)   
2       CO  PUB  22.0 -104.8400  38.7500  POINT (-420053.206 -682413.408)   
3       NM  ABQ  21.0 -106.2500  36.7000  POINT (-558038.711 -901349.509)   
4       NM  ABQ  21.0 -105.3700  36.5300  POINT (-480721.184 -925612.543)   
...    ...  ...   ...       ...      ...                              ...   
4810    TX  LUB   0.0 -100.3369  33.6266  POINT (-31347.067 -1262526.539)   
4814    TX  LUB   0.0 -100.0859  34.3906   POINT (-7915.974 -1178022.229)   
4817    TX  LUB   0.0 -100.5378  34.7259  POINT (-49347.120 -1140740.878)   
4823    TX  LUB   0.0 -100.3878  33.8769  POINT (-35970.134 -1234808.482)   
4824    TX  LUB   0.0 -100.2980  33.8084  POINT (-27664.624 -1242422.761)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0            